In [ ]:
import pandas as pd
import numpy as np
from gensim.summarization.summarizer import summarize 
from gensim.summarization import keywords
import spacy
import re           
import nltk
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
data=pd.read_csv('/content/drive/My Drive/Dusra app/Dusra App/Task 2- Dataset/Games/games_df.csv', encoding='iso-8859-1')

In [ ]:
data['cleaned_content']=data['content']
data=data.dropna()

In [ ]:
data.head()

,reviewId,userName,content,score,thumbsUpCount,at,Sort_Order,App_Id,cleaned_content
0,gp:AOqpTOE-VFQ0bM0LerLxxZmEBkfWosnc1SxPs8RjLLa...,rummy vinay,I've playing this game for more than a year an...,4,3851,2020-06-21 08:13:05,most_relevant,com.dts.freefireth,I've playing this game for more than a year an...
1,gp:AOqpTOFg8Jv_Ok57T2Ws9urYGges44c3f829S0ZcCA3...,Luigi Tolosa,I think this game would be better if the serve...,3,3386,2020-06-21 04:31:56,most_relevant,com.dts.freefireth,I think this game would be better if the serve...
2,gp:AOqpTOGWnNWDeWanc3CEr0QOgFgZsjOfgKB6AKxGUoL...,PHUKHAZ .M.P,The game is really good to play even though th...,4,3107,2020-06-21 06:13:18,most_relevant,com.dts.freefireth,The game is really good to play even though th...
3,gp:AOqpTOExoh7P_S2Izd0lG08Q4iNWW7Qbkxe8CiyzDwu...,UNBELIVEABLE TEACHING,It's a good type of game and had a good time p...,3,744,2020-06-19 13:15:05,most_relevant,com.dts.freefireth,It's a good type of game and had a good time p...
4,gp:AOqpTOERYczBuHch65EV1qzYRUl1PSgf1CQ6yXZBoKc...,theo kingdom,"Great game but kind of a wallet warriors game,...",3,549,2020-06-19 22:06:20,most_relevant,com.dts.freefireth,"Great game but kind of a wallet warriors game,..."


In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}


In [ ]:
import re
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
data['cleaned_content']=(data['content'].astype(str)).apply(lambda x:remove_emoji(x))

In [ ]:
stop_words = set(stopwords.words('english')) 
def text_cleaner(text):
  newString=text
  newString = BeautifulSoup(newString, "lxml").text
  newString = re.sub(r'\([^)]*\)', '', newString)
  newString = re.sub('"','', newString)
  newString = text.lower()
  newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
  newString = re.sub(r"'s\b","",newString)
  newString = re.sub("[^a-zA-Z]", " ", newString) 
  tokens = [w for w in newString.split() if not w in stop_words]
  long_words=[]
  for i in tokens:
    if len(i)>=3:             
      long_words.append(i)   
  return (" ".join(long_words)).strip()

cleaned_text = []
for t in data['cleaned_content']:
    cleaned_text.append(text_cleaner(t))
data['cleaned_content']=cleaned_text

In [ ]:
data.head()

,reviewId,userName,content,score,thumbsUpCount,at,Sort_Order,App_Id,cleaned_content
0,gp:AOqpTOE-VFQ0bM0LerLxxZmEBkfWosnc1SxPs8RjLLa...,rummy vinay,I've playing this game for more than a year an...,4,3851,2020-06-21 08:13:05,most_relevant,com.dts.freefireth,playing game year still purchased one elite pa...
1,gp:AOqpTOFg8Jv_Ok57T2Ws9urYGges44c3f829S0ZcCA3...,Luigi Tolosa,I think this game would be better if the serve...,3,3386,2020-06-21 04:31:56,most_relevant,com.dts.freefireth,think game would better server fix network con...
2,gp:AOqpTOGWnNWDeWanc3CEr0QOgFgZsjOfgKB6AKxGUoL...,PHUKHAZ .M.P,The game is really good to play even though th...,4,3107,2020-06-21 06:13:18,most_relevant,com.dts.freefireth,game really good play even though graphics mar...
3,gp:AOqpTOExoh7P_S2Izd0lG08Q4iNWW7Qbkxe8CiyzDwu...,UNBELIVEABLE TEACHING,It's a good type of game and had a good time p...,3,744,2020-06-19 13:15:05,most_relevant,com.dts.freefireth,good type game good time playing experience al...
4,gp:AOqpTOERYczBuHch65EV1qzYRUl1PSgf1CQ6yXZBoKc...,theo kingdom,"Great game but kind of a wallet warriors game,...",3,549,2020-06-19 22:06:20,most_relevant,com.dts.freefireth,great game kind wallet warriors game pay gems ...


In [ ]:
data=data.dropna()

In [ ]:
dataset=[]
temp=list(data['cleaned_content'])
dataset=[TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(temp)]

In [ ]:
data['cleaned_content']

In [ ]:
data.isna().sum()

reviewId           0
userName           0
content            0
score              0
thumbsUpCount      0
at                 0
Sort_Order         0
App_Id             0
cleaned_content    0
dtype: int64

In [ ]:
len(dataset)

19999

In [ ]:
max_epochs = 100
vec_size = 20
alpha = 0.025

In [ ]:
model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [ ]:
model.build_vocab(dataset)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(dataset,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

iteration 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  import sys


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [ ]:
model.save("d2v1.model")
print("Model Saved")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Model Saved


In [ ]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v1.model")


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
text="Tgood game"
text=remove_emoji(text)
text=text_cleaner(text)
text=word_tokenize(text.lower())
text

['tgood', 'game']

In [ ]:
vector=model.infer_vector(text)
index = model.docvecs.most_similar([vector],topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print(index)

[('11621', 0.7958335280418396), ('222', 0.7467595338821411), ('11094', 0.7314677238464355), ('4558', 0.7221856713294983), ('194', 0.7219529151916504), ('1455', 0.7200127840042114), ('12362', 0.7090641856193542), ('8351', 0.6954125165939331), ('9580', 0.6917666792869568), ('3041', 0.6903694272041321)]


In [ ]:
res=[]
for i in range(len(index)):
  res.append(data['content'][int(index[i][0])])

In [ ]:
res

['Enjoyed a lot this game. 1000 times better than PubG',
 "This is a pay to win game, the game used to be fun but the events they give are more shady then a Las Vegas Casino, hacker issue has been bothering the player base for a long time, also the games connection issue is big thing, one moment you have 50ms and instantly get 300+ms while using WiFi and if you thing WiFi is slow I play Warzone all the time and get 60ms average, if you don't want to get addicted and spend huge amount of money and get kicked out of your house then don't play this F game",
 'Why do people complain about there devices problem and not the game',
 'I cant recover my account and too many ads.',
 "I have been playing this game for over a 2 years now. The only thing I doesn't like is the network problem and only the elite pass holders are getting rare items and other are not. So whatever the items like bundles or gun skins and other items, just give to players by challenges, so that they may encouraged. #Thank

In [ ]:
index

[('11621', 0.7958335280418396),
 ('222', 0.7467595338821411),
 ('11094', 0.7314677238464355),
 ('4558', 0.7221856713294983),
 ('194', 0.7219529151916504),
 ('1455', 0.7200127840042114),
 ('12362', 0.7090641856193542),
 ('8351', 0.6954125165939331),
 ('9580', 0.6917666792869568),
 ('3041', 0.6903694272041321)]

ye above 10 ko summary create karna h , extractive wala
